**Homework 3 - Part B**


Jade Benson

In [1]:
import pandas as pd 
import numpy as np 
import statistics as stats

import sklearn
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing, datasets, metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_validate, train_test_split, cross_val_predict,  cross_val_score
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_auc_score, mean_squared_error, r2_score

#random forest 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import make_classification

#for importing files

import os
import glob


In this problem, we are interested in developing 53 separate models that use gene expression data to predict each of the 53 drugs' efficacy. I chose random forests since I think these are the most intutive and interpretable models which may be important in explaining these results to doctors, pharmaceutical companies, patients, etc. There are also random forest models for both classification and regression, which simplifies this problem. I'll use the classifier for the SIR problem and the regressor for the regression problem. I've binarized the SIR problem so it's now just an SR problem. I think this approach better handles the data imbalances. I tried this with three classes (thresholds at thirds) and the random forest rarely categorized the smallest group, which made it difficult to even calculate AUC scores. Although as we'll see, even the F1 scores and AUC values for this binary problem are very low.

Hopefully the regression approach will do a better job at handling this problem. 



In [2]:
filepaths = ["MLiB-Lab3-PartB/Data_Frames/" + f for f in os.listdir("MLiB-Lab3-PartB/Data_Frames/") if f.endswith('.tsv')]

In [143]:
drug_scores = []


In [147]:
def classify_drugs(file_path, drug_scores):
    df = pd.read_csv(file_path, sep='\t')
    
    #create binary classifier column at median
    df["SIR"] = np.where(
        df["AUC1"] <= stats.median(df["AUC1"]), 0, 1)
    
    #create y and x dataframes 
    y_df = df["SIR"]
    x_df = df.drop(columns = ["CELL", "DRUG", "AUC1", "SIR"])
    
    #run random forest classifier
    rf_pipe = make_pipeline(StandardScaler(), RandomForestClassifier())

    #predict x and y using 5-fold cross validation
    y_pred_rf = cross_val_predict(rf_pipe, x_df, y_df, cv = 5)

    #use drug name, f1 score, and ROC_AUC to rank each drug's model
    drug_name = df["DRUG"][0]
    f1 = f1_score(y_df, y_pred_rf)
    auc = roc_auc_score(y_df, y_pred_rf)
    
    drug_info = [drug_name, f1, auc]
    
    drug_scores.append(drug_info)
    


In [148]:
for file in filepaths: 
    classify_drugs(file, drug_scores)

Since this is a drug identification problem, we are most interested in the true positive rate (F1 score). We want to minimize the false positive rate first as this may cause cancer patients to pursue unhelpful treatments and waste the resource they value most - time. I will rank all the models for each of the drugs by their F1 scores first and their AUC values second and report the top 10 best models and the worst 10. 

In [156]:
drug_model_performances = pd.DataFrame(drug_scores, columns = ['Drug_Name', 'F1_Score', 'AUC_ROC'])

In [157]:
ranked_drug_models = drug_model_performances.sort_values(by=['F1_Score', 'AUC_ROC'], ascending = False)
#all models ranked by F1 score and AUC score 
ranked_drug_models

,Drug_Name,F1_Score,AUC_ROC
37,CTRP.29,0.903687,0.900061
31,CTRP.343,0.899303,0.897683
22,CTRP.33,0.891693,0.883368
28,CTRP.414,0.763018,0.730676
42,CTRP.487,0.746988,0.733703
23,CTRP.413,0.746554,0.720838
50,CTRP.318,0.731159,0.704950
14,CTRP.91,0.727273,0.716216
56,CTRP.441,0.725843,0.687760
35,CTRP.226,0.724968,0.702683


In [159]:
#Top 10 drug models 
ranked_drug_models[0:9]

,Drug_Name,F1_Score,AUC_ROC
37,CTRP.29,0.903687,0.900061
31,CTRP.343,0.899303,0.897683
22,CTRP.33,0.891693,0.883368
28,CTRP.414,0.763018,0.730676
42,CTRP.487,0.746988,0.733703
23,CTRP.413,0.746554,0.720838
50,CTRP.318,0.731159,0.704950
14,CTRP.91,0.727273,0.716216
56,CTRP.441,0.725843,0.687760


There are a few where these models perform well (best one has an F1 and an AUC of 90%). However, even the tenth best model has a 28% false positive rate (1 - F1) which is really high and could waste a lot of time and resources. The AUC value that considers both true positives and negatives is only about 70% which isn't great. 

There may be other factors that explain a drug's efficacy beyond just the tumor's gene composition. As we saw in class, the top of the line models take into account many features of the drug itself and other aspects of gene expression data. We might want to add other data sources, address data imbalances, choose a different model type for different drugs, and/or tune hyperparameters to improve our models' performances. 

In [162]:
#Worst 10 drug models 
ranked_drug_models[-10:]

,Drug_Name,F1_Score,AUC_ROC
10,CTRP.539,0.532110,0.528887
32,CTRP.155,0.521739,0.552632
12,CTRP.7,0.521401,0.533812
24,CTRP.163,0.512195,0.532915
43,CTRP.64,0.492754,0.519895
34,CTRP.395,0.490566,0.502383
53,CTRP.89,0.473430,0.501293
17,CTRP.305,0.459770,0.477778
19,CTRP.80,0.414414,0.453249
41,CTRP.323,0.407143,0.462880


As we saw even the best models weren't great. The worst models demonsrate that some of these models are worse than change alone in guessing positives since this is a binary problem (last six models have f1 < 0.5). There would certainly need to be improvements made to accurately model these drugs' efficacy in different tumors

*Regression Approach* 

Now we will examine this problem as a regression problem where we continously predict the drug's efficacy with the AUC1 value. 

In [167]:
#create y and x dataframes 
train_code_y = df_470["AUC1"]
train_code_x = df_470.drop(columns = ["CELL", "DRUG", "AUC1"])

In [174]:
#run random forest classifier
rfr_pipe = make_pipeline(StandardScaler(), RandomForestRegressor())

#predict x and y using 5-fold cross validation
y_pred_rfr = cross_val_predict(rfr_pipe, train_code_x, train_code_y, cv = 5)

#use drug name, f1 score, and ROC_AUC to rank each drug's model
#drug_name = df["DRUG"][0]
r2 = r2_score(train_code_y, y_pred_rfr)
print(r2)

mse = mean_squared_error(train_code_y, y_pred_rfr)
print(mse)

0.624171217785698
0.00720543935676242


In [183]:
regress_drug_scores = []

In [186]:
def regress_drugs(file_path, regress_drug_scores):
    df = pd.read_csv(file_path, sep='\t')
    
    #create y and x dataframes 
    y_df = df["AUC1"]
    x_df = df.drop(columns = ["CELL", "DRUG", "AUC1"])
    
    #run random forest regressor
    rfr_pipe = make_pipeline(StandardScaler(), RandomForestRegressor())

    #predict x and y using 5-fold cross validation
    y_pred_rfr = cross_val_predict(rfr_pipe, x_df, y_df, cv = 5)

    #use drug name, f1 score, and ROC_AUC to rank each drug's model
    drug_name = df["DRUG"][0]
    r2 = r2_score(y_df, y_pred_rfr)
    mse = mean_squared_error(y_df, y_pred_rfr)
    
    drug_info = [drug_name, r2, mse]
    
    #keep track of status since this runs slower
    print(drug_info)
    
    regress_drug_scores.append(drug_info)
    


In [188]:
for file in filepaths: 
    regress_drugs(file, regress_drug_scores)

['CTRP.470', 0.17866335906223385, 0.01574677522805972]
['CTRP.464', -0.03472515194793213, 0.001211243644351847]
['CTRP.101', 0.11742427540386124, 0.0008101634817993864]
['CTRP.303', 0.04667613606018328, 0.004293617884949091]
['CTRP.505', -0.06400454006604694, 0.002463684079819453]
['CTRP.539', -0.04072237567984538, 0.001894787685980105]
['CTRP.51', 0.11909114655358721, 0.007298934933712041]
['CTRP.7', -0.09849541141685192, 0.0014656065854887607]
['CTRP.472', 0.2228918879023113, 0.017011418121111097]
['CTRP.91', -0.10220673108856837, 0.0017555933112975018]
['CTRP.2', 0.13856637437334118, 0.016656480444642578]
['CTRP.258', 0.1062358493100799, 0.010511624647942761]
['CTRP.305', -0.275468846183875, 0.001397760169294826]
['CTRP.271', 0.24706992762218982, 0.012344993526560802]
['CTRP.80', -0.03948860728529513, 0.002626822577998433]
['CTRP.313', 0.1147113253520553, 0.0028629002188967616]
['CTRP.474', 0.19810009407601947, 0.02543588593965115]
['CTRP.33', 0.6493353892930592, 0.00657517015231353

In [191]:
regress_drug_model_performances = pd.DataFrame(regress_drug_scores, columns = ['Drug_Name', 'F1_Score', 'AUC_ROC'])
regress_ranked_drug_models = regress_drug_model_performances.sort_values(by=['F1_Score', 'AUC_ROC'], ascending = False)
#all models ranked by F1 score and AUC score 
regress_ranked_drug_models

,Drug_Name,F1_Score,AUC_ROC
32,CTRP.29,0.680162,0.007347
17,CTRP.33,0.649335,0.006575
26,CTRP.343,0.630324,0.017172
28,CTRP.196,0.450972,0.024325
23,CTRP.414,0.431848,0.018261
31,CTRP.193,0.362461,0.005228
18,CTRP.413,0.327740,0.016337
49,CTRP.285,0.319417,0.010551
33,CTRP.345,0.315873,0.016318
51,CTRP.441,0.310167,0.020552


In [192]:
#Top 10 drug models 
regress_ranked_drug_models[0:9]

,Drug_Name,F1_Score,AUC_ROC
32,CTRP.29,0.680162,0.007347
17,CTRP.33,0.649335,0.006575
26,CTRP.343,0.630324,0.017172
28,CTRP.196,0.450972,0.024325
23,CTRP.414,0.431848,0.018261
31,CTRP.193,0.362461,0.005228
18,CTRP.413,0.327740,0.016337
49,CTRP.285,0.319417,0.010551
33,CTRP.345,0.315873,0.016318


These regression models perform worse than the classification models. This may be because random forest regression is not well-suited to this problem or should be better adjusted for each case. This decrease in true positive rate may be because the problem has gotten more complicated than just predicting binary values. Only the top few cases have moderate values and the rest are not very useful. This may also just be a result of bad statistics to measure these model performances since these metrics are more geared towards classification and are averaged for regression problems. 

As we see with the worst 10 models, this approach is not at all useful. Future work is definitely neeeded to improve these results. 

In [194]:
#Worst 10 drug models 
regress_ranked_drug_models[-10:]

,Drug_Name,F1_Score,AUC_ROC
29,CTRP.395,-0.054874,0.002413
4,CTRP.505,-0.064005,0.002464
38,CTRP.64,-0.087153,0.001460
7,CTRP.7,-0.098495,0.001466
9,CTRP.91,-0.102207,0.001756
36,CTRP.323,-0.125000,0.002272
52,CTRP.75,-0.143412,0.002461
19,CTRP.163,-0.199490,0.004988
27,CTRP.155,-0.223447,0.001114
12,CTRP.305,-0.275469,0.001398


*Question 2*

In [3]:
first_df = pd.read_csv(filepaths[0], sep='\t')
#rename AUC column after the drug, drop DRUG 
drug_name = first_df["DRUG"][0]
df_start = first_df.rename(columns={"AUC1": drug_name})

df_start = df_start[["CELL", drug_name]]


In [4]:
def merge_cell_AUC(file_path, df_old):
    
    df = pd.read_csv(file_path, sep='\t')
    
    #rename AUC column after the drug, drop DRUG 
    drug_name = df["DRUG"][0]
    df_new = df.rename(columns={"AUC1": drug_name})

    df_new = df_new[["CELL", drug_name]]

    #merge together all dataframes by CELL line
    return df_old.merge(df_new, how = "outer", on = "CELL")
    


I apologize, my computer had a very difficult time performing this merge between all 53 dataframes. I'm not sure how else I would get around this step to be able to develop a model that predicts the best drug for each cell line. 

I was thinking that the most basic way to decide the top 10 performing drugs on each cell line would just be rank ordering the AUC values. If we wanted to perform a machine learning approach, I would re-write my code in question 1 to save each of the 53 drug response regression models so that I could predict the AUC values and then compare each of these models to each other. Honestly, I'm not sure how much is gained by that approach as these models were already performing so poorly to begin with. If we have the true efficacy in the AUC1 values shouldn't we just rank this to determine the best drugs for each of the cell lines? I would be interested to discuss this problem more with you since I think I'm missing something. 